# Disclaimer & Copyright
Copyright 2024 Forusone : hangsik@google.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Basic operation with Gemini in dealing with only text
The purpose of this notebook is to test the scenarios


In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform
%pip install --upgrade --quiet google-cloud-discoveryengine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00


In [ ]:
from IPython.display import display, Markdown

In [ ]:
#  For only colab
import sys

if "google.colab" in sys.modules:
    # to authenticate to get an access to the GCP.
    # from google.colab import auth
    # auth.authenticate_user()

    # To access contents in Google drive
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

# Set the working directory to system path.
src_dir = '/content/drive/MyDrive/projects/rag_example/'
sys.path.append(src_dir)
os.chdir(src_dir)

# print current directory to check the source root directory.
!pwd


/content/drive/MyDrive/projects/rag_example


In [ ]:
%load_ext autoreload
%autoreload 2

# Authentication setting to invoke Cloud Functions

In [ ]:
!gcloud auth application-default login
!gcloud config set project ai-hangsik
!gcloud auth application-default set-quota-project ai-hangsik

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=nvBAeqmPZGyrIcsOmG847gCP9Lz8VS&prompt=consent&token_usage=remote&access_type=offline&code_challenge=aVBSGXzjMxDlqiXcmqKDQqq0efUFJtoPPNVKSbjLLt0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ATx3LY6sWQCmbaVijc77t5-r0GJslQ48tPl5JWvuU0yXMvQvbAsPHDI5ey2u9v7wyEKWcA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
import vertexai.generative_models as generative_models

PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

# MODEL_NAME="gemini-1.5-pro-001"
MODEL_NAME="gemini-1.5-flash-001"

# MODEL_NAME="gemini-experimental"
# REGION="us-central1"

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)
# model = GenerativeModel(MODEL_NAME)

In [ ]:
generation_config = {
  "max_output_tokens": 8192,
  "temperature": 1,
  "top_p": 0.7,
  "top_k": 30,
}

system_instruction = [
 "당신은 상품 정보를 상담해주는 AI 어시스턴트 입니다.",
 "Prompt 상의 Context 정보 내용으로만으로 한국어로 답을 해주세요.",
 "만일 Context 정보로 답을 하기 어려우면 '현재 콘텐츠에는 정보가 없습니다.' 라고 말해주세요.",
]

model = GenerativeModel(MODEL_NAME,
                        generation_config = generation_config,
                        system_instruction = system_instruction)

In [9]:

import json
import os
import requests

import google.auth
import google.auth.transport.requests

credentials, project = google.auth.default()
request = google.auth.transport.requests.Request()

credentials.refresh(request)
credential_identity_token = credentials.id_token

In [11]:
import time
import asyncio
import aiohttp

async def make_request(url, payload):

  headers = {
      "Authorization": "Bearer "+ credential_identity_token,
      "Content-Type": "application/json"
  }

  results = []

  async with aiohttp.ClientSession() as client:
      async with client.post(url, headers=headers, data=payload) as resp:
          response = await resp.text()
          results.append(response)
          print(resp.status)

  return results


In [10]:
async def search_text(query):

  #------------ Text Search ------------
  TEXT_URL = 'https://asia-northeast3-ai-hangsik.cloudfunctions.net/text_process'
  TEXT_SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/rubicon-text-search-ds_1719010635403/servingConfigs/default_search:search"

  text_payload_dict = {}
  text_payload_dict['query'] = query
  text_payload_dict['num_query_rewrite'] = 2
  text_payload_dict['search_type'] = "LAYOUT_SEARCH"
  text_payload_dict['search_url'] = TEXT_SEARCH_URL
  text_payload_dict['page_size'] = 2

  text_data = json.dumps(text_payload_dict)
  text_payload = text_data.encode("utf8")

  #------------ PDF Image Search ------------

  IMAGE_URL = 'https://asia-northeast3-ai-hangsik.cloudfunctions.net/image_process'
  IMAGE_SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/rubicon-bq-search-ds_1719649178491/servingConfigs/default_search:search"
  image_payload_dict = {}
  image_payload_dict['query'] = query
  image_payload_dict['num_query_rewrite'] = 2
  image_payload_dict['search_type'] = "BQ_SEARCH"
  image_payload_dict['search_url'] = IMAGE_SEARCH_URL
  image_payload_dict['page_size'] = 1

  image_data = json.dumps(image_payload_dict)
  image_payload = image_data.encode("utf8")

  #------------ Google Search ------------

  SEARCH_URL = 'https://asia-northeast3-ai-hangsik.cloudfunctions.net/google_search_process'
  search_payload_dict = {}
  search_payload_dict['query'] = query
  search_payload_dict['search_type'] = "GOOGLE_SEARCH"

  search_data = json.dumps(search_payload_dict)
  search_payload = search_data.encode("utf8")

  tasks = []
  # for url in urls:
  tasks.append(asyncio.create_task(make_request(url=TEXT_URL, payload=text_payload )))
  tasks.append(asyncio.create_task(make_request(url=IMAGE_URL, payload=image_payload )))
  tasks.append(asyncio.create_task(make_request(url=SEARCH_URL, payload=search_payload )))

  final_results_list = await asyncio.gather(*tasks)

  t2 = time.perf_counter()

  search_elapsed_time = round((t2 - t1),3)
  print(f"search_elapsed_time:{search_elapsed_time}")

  final_contexts_dict = {}
  for idx, final_result in enumerate(final_results_list):
    final_contexts_dict[f'Context-{idx}'] = "["+eval(final_result[0])['search_type'] + "]" +eval(final_result[0])['final_outcome']

  final_contexts = json.dumps(final_contexts_dict, ensure_ascii=False )

  print(f"final_contexts : {final_contexts}")

  return final_contexts, search_elapsed_time


In [22]:
def existed_history(query:str, contexts:str):

  generation_config = GenerationConfig(
      max_output_tokens = 8192,
      temperature = 1,
      top_p = 0.95
  )

  prompt = f"""
    당신은 질문에 답하기 위해서 Context 내의 해당 정보가 있는지 확인하는 AI입니다.
    만일 질문 내용에 대해서 아래 Context 정보를 기반으로 답을 할 수 있을 경우 'Yes' 로, 아니면 'No' 로 답해주세요.
    <Context> {contexts}</Context>
    <Question> {query}</Question>

    만일 질문 내용에 아래 내용의 의미가 있으면 'No' 로 답해주세요.
    1. 좀더 자세하게 설명해주세요.
    2. 다른 정보를 알려주세요.
    3. 상세하게 설명해주세요.

    답변포맷 : 'Yes' or 'No'
  """

  responses = model.generate_content(
      prompt,
      generation_config=generation_config,
      stream=False,
  )

  print(f"responses.text : {responses.text}")

  return responses.text

* https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession


In [23]:
from vertexai.generative_models import ChatSession

async def interactive_chat(chat:ChatSession,
                      query:str,
                      history)->str:

  search = existed_history(query, history)
  print(f"search:[{search}]")

  final_contexts = ""

  if search.strip() == "No":
    final_contexts, search_elapsed_time = await search_text(query)
    print(f"final_contexts : {final_contexts}")

  prompt = f"""
    질문에 맞게 아래 <Context> 내용을 참고해서 답을 해주세요.
    <Context> {final_contexts} {history}</Context>
    <Question> {query}</Question>
  """

  responses = chat.send_message(
      prompt,
      tools = None,
      stream=False
  )

  return responses.text

In [24]:
import json
import pandas as pd

q1 = "Galaxy S24의 특징을 설명해주세요."
q2 = "Galaxy S23의 특징을 설명해주세요."
q3 = "갤럭시 S24U의 CPU의 사양을 알려주세요."
q4 = "갤럭시 S24는 언제 출시해?"
q5 = "갤럭시 S23는 출시일이 언제야?"
q6 = "갤럭시 S24의 Circle to Search가 무슨 기능인가요?"
q7 = "S24의 실시간 통역된 대화 내용은 서버에 수집되는건가요?"
q8 = "내가 사진을 좋아해서 사진을 많이 저장하는데, 갤럭시 S23 Ultra 과 갤럭시 S24+ 중 어느 게 더 적합할까?"
q9 = "S24 시리즈 간에 카메라 사양이 어떻게 다른가요? 표로 좀 보기좋게 정리해 줄래요?"
q10 = "스마트폰 SM-S711NZOWKOO과 SM-S921NZAFKOO 간에 어떻게 다른가요?"
q11 = "S24 실시간 통역 기능은 어떻게 사용하는건가요?"
q12 = "앱 쿠폰, 카드 할인 혜택 적용했을 시 갤럭시 S24 최저가 알려줘"


In [25]:
import time


t1 = time.perf_counter()

chat = model.start_chat()

t2 = time.perf_counter()

history = "History: ".join([content for content in chat.history])
print(history)

query = "Galaxy S24의 특징을 설명해주세요."
answer = await interactive_chat(chat, query, history)

display(Markdown(answer))

t3 = time.perf_counter()

print(f"Chat start : {round(t2-t1,3)}")

print(f"Elapsed time : {round(t3-t2,3)}")



responses.text : No

search:[No
]
200
200
200
search_elapsed_time:10.24
final_contexts : {"Context-0": "[LAYOUT_SEARCH]해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n", "Context-1": "[BQ_SEARCH]Context 내에 해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n Galaxy S24 시리즈는 삼성의 지속 가능한 목표를 반영하여 친환경적인 기능을 갖추고 있습니다. 2030년까지 모든 모바일 제품의 모든 모듈에 재활용 소재를 사용한다는 목표를 가지고 있으며, S24 시리즈는 이러한 노력의 일환으로 다음과 같은 친환경적인 특징을 가지고 있습니다.\n\n* **재활용 소재 사용:** S24 시리즈는 해양 플라스틱, 유리, 알루미늄과 같은 재활용 소재를 사용합니다. 또한 S 시리즈 최초로 재활용 코발트, 희토류 원소, 강철, 소비 전 재활용 TPU를 사용합니다.\n* **재활용 포장:** S24 시리즈의 포장 상자는 100% 재활용 종이로 만들어졌습니다. 이는 일회용 플라스틱 사용을 줄이기 위한 노력의 일환입니다.\n* **제품 수명 연장:** S24 시리즈는 7세대 OS 업그레이드와 7년간의 보안 업데이트를 제공하여 제품 수명 주기를 연장합니다.\n* **내구성 강화:** 내구성이 뛰어난 S24 Ultra는 Corning® Gorilla® Armor로 강화되어 긁힘 방지 기능이 강화되었습니다.\n\n이러한 친환경적인 특징을 통해 Galaxy S24 시리즈는 지속 가능한 미래를 위한 삼성의 노력을 보여주는 좋은 예시가 됩니다.\n", "Context-2": "[GOOGLE_SEARCH]Samsung Galaxy S24는 120Hz 재생률을 갖춘 6.2인치 Dynamic AMOLED 디스플레이를 갖추고 있습니다. Qualcomm Snapdragon 8 Gen 3 프로세서로 구동되며 8GB RAM을 갖추고 있습니다. 이 전화

Galaxy S24는 120Hz 재생률을 갖춘 6.2인치 Dynamic AMOLED 디스플레이를 갖추고 있습니다. Qualcomm Snapdragon 8 Gen 3 프로세서로 구동되며 8GB RAM을 갖추고 있습니다. 이 전화기에는 후면에 50MP 메인 카메라와 4000mAh 배터리를 갖춘 트리플 카메라 설정이 있습니다. Galaxy S24는 5G를 지원하며 무선 충전을 지원합니다. 화면 대 본체 비율도 90.87%로 높습니다. 


Chat start : 0.0
Elapsed time : 11.979


In [26]:
history = "History: ".join([content.text for content in chat.history])
print(history)

query = "디스플레이는 어떤가요 ?"
answer = await interactive_chat(chat, query, history )
print(f"answer : {answer}")


    질문에 맞게 아래 <Context> 내용을 참고해서 답을 해주세요.
    <Context> {"Context-0": "[LAYOUT_SEARCH]해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n", "Context-1": "[BQ_SEARCH]Context 내에 해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n Galaxy S24 시리즈는 삼성의 지속 가능한 목표를 반영하여 친환경적인 기능을 갖추고 있습니다. 2030년까지 모든 모바일 제품의 모든 모듈에 재활용 소재를 사용한다는 목표를 가지고 있으며, S24 시리즈는 이러한 노력의 일환으로 다음과 같은 친환경적인 특징을 가지고 있습니다.\n\n* **재활용 소재 사용:** S24 시리즈는 해양 플라스틱, 유리, 알루미늄과 같은 재활용 소재를 사용합니다. 또한 S 시리즈 최초로 재활용 코발트, 희토류 원소, 강철, 소비 전 재활용 TPU를 사용합니다.\n* **재활용 포장:** S24 시리즈의 포장 상자는 100% 재활용 종이로 만들어졌습니다. 이는 일회용 플라스틱 사용을 줄이기 위한 노력의 일환입니다.\n* **제품 수명 연장:** S24 시리즈는 7세대 OS 업그레이드와 7년간의 보안 업데이트를 제공하여 제품 수명 주기를 연장합니다.\n* **내구성 강화:** 내구성이 뛰어난 S24 Ultra는 Corning® Gorilla® Armor로 강화되어 긁힘 방지 기능이 강화되었습니다.\n\n이러한 친환경적인 특징을 통해 Galaxy S24 시리즈는 지속 가능한 미래를 위한 삼성의 노력을 보여주는 좋은 예시가 됩니다.\n", "Context-2": "[GOOGLE_SEARCH]Samsung Galaxy S24는 120Hz 재생률을 갖춘 6.2인치 Dynamic AMOLED 디스플레이를 갖추고 있습니다. Qualcomm Snapdragon 8 Gen 3 프로세서로 구동되며 8GB RAM을 갖추고 있습니다. 이 전화기에는 후면에 50MP 메인 카메라와 4000mAh 배터리를

In [27]:
history = "History: ".join([content.text for content in chat.history])
print(history)

query = "Gorilla Glass는 무엇인가요 ?"
answer = await interactive_chat(chat, query, history )
print(f"answer : {answer}")


    질문에 맞게 아래 <Context> 내용을 참고해서 답을 해주세요.
    <Context> {"Context-0": "[LAYOUT_SEARCH]해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n", "Context-1": "[BQ_SEARCH]Context 내에 해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n Galaxy S24 시리즈는 삼성의 지속 가능한 목표를 반영하여 친환경적인 기능을 갖추고 있습니다. 2030년까지 모든 모바일 제품의 모든 모듈에 재활용 소재를 사용한다는 목표를 가지고 있으며, S24 시리즈는 이러한 노력의 일환으로 다음과 같은 친환경적인 특징을 가지고 있습니다.\n\n* **재활용 소재 사용:** S24 시리즈는 해양 플라스틱, 유리, 알루미늄과 같은 재활용 소재를 사용합니다. 또한 S 시리즈 최초로 재활용 코발트, 희토류 원소, 강철, 소비 전 재활용 TPU를 사용합니다.\n* **재활용 포장:** S24 시리즈의 포장 상자는 100% 재활용 종이로 만들어졌습니다. 이는 일회용 플라스틱 사용을 줄이기 위한 노력의 일환입니다.\n* **제품 수명 연장:** S24 시리즈는 7세대 OS 업그레이드와 7년간의 보안 업데이트를 제공하여 제품 수명 주기를 연장합니다.\n* **내구성 강화:** 내구성이 뛰어난 S24 Ultra는 Corning® Gorilla® Armor로 강화되어 긁힘 방지 기능이 강화되었습니다.\n\n이러한 친환경적인 특징을 통해 Galaxy S24 시리즈는 지속 가능한 미래를 위한 삼성의 노력을 보여주는 좋은 예시가 됩니다.\n", "Context-2": "[GOOGLE_SEARCH]Samsung Galaxy S24는 120Hz 재생률을 갖춘 6.2인치 Dynamic AMOLED 디스플레이를 갖추고 있습니다. Qualcomm Snapdragon 8 Gen 3 프로세서로 구동되며 8GB RAM을 갖추고 있습니다. 이 전화기에는 후면에 50MP 메인 카메라와 4000mAh 배터리를

In [29]:
history = "History: ".join([content.text for content in chat.history])
print(history)

query = "지금 이야기하고 있는 제품 이름을 알려주세요."
answer = await interactive_chat(chat, query, history )
print(f"answer : {answer}")


    질문에 맞게 아래 <Context> 내용을 참고해서 답을 해주세요.
    <Context> {"Context-0": "[LAYOUT_SEARCH]해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n", "Context-1": "[BQ_SEARCH]Context 내에 해당 컨텍스트가 없습니다. 좀더 자세한 정보를 제공해주세요. \n Galaxy S24 시리즈는 삼성의 지속 가능한 목표를 반영하여 친환경적인 기능을 갖추고 있습니다. 2030년까지 모든 모바일 제품의 모든 모듈에 재활용 소재를 사용한다는 목표를 가지고 있으며, S24 시리즈는 이러한 노력의 일환으로 다음과 같은 친환경적인 특징을 가지고 있습니다.\n\n* **재활용 소재 사용:** S24 시리즈는 해양 플라스틱, 유리, 알루미늄과 같은 재활용 소재를 사용합니다. 또한 S 시리즈 최초로 재활용 코발트, 희토류 원소, 강철, 소비 전 재활용 TPU를 사용합니다.\n* **재활용 포장:** S24 시리즈의 포장 상자는 100% 재활용 종이로 만들어졌습니다. 이는 일회용 플라스틱 사용을 줄이기 위한 노력의 일환입니다.\n* **제품 수명 연장:** S24 시리즈는 7세대 OS 업그레이드와 7년간의 보안 업데이트를 제공하여 제품 수명 주기를 연장합니다.\n* **내구성 강화:** 내구성이 뛰어난 S24 Ultra는 Corning® Gorilla® Armor로 강화되어 긁힘 방지 기능이 강화되었습니다.\n\n이러한 친환경적인 특징을 통해 Galaxy S24 시리즈는 지속 가능한 미래를 위한 삼성의 노력을 보여주는 좋은 예시가 됩니다.\n", "Context-2": "[GOOGLE_SEARCH]Samsung Galaxy S24는 120Hz 재생률을 갖춘 6.2인치 Dynamic AMOLED 디스플레이를 갖추고 있습니다. Qualcomm Snapdragon 8 Gen 3 프로세서로 구동되며 8GB RAM을 갖추고 있습니다. 이 전화기에는 후면에 50MP 메인 카메라와 4000mAh 배터리를